In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
rawContests = pd.read_csv('data/WorkingData.csv').set_index('ContestId')
expensiveContests = rawContests.loc[rawContests['TotalPrizeAmount']> 100000][[]]

In [3]:
contests = pd.read_csv('data/SCALEDEVERYTHING.csv').set_index('ContestId')
contests.head(1)

,Success,SportName_NFL,SportName_PGA,SportName_SOCC,SportName_MLB,SportName_NAS,SportName_CFB,SportName_MMA,SportName_LOL,SportName_NHL,...,2HoursOut_scaled,36HoursOut_scaled,3HoursOut_scaled,48HoursOut_scaled,4HoursOut_scaled,5HoursOut_scaled,6HoursOut_scaled,7HoursOut_scaled,8HoursOut_scaled,9HoursOut_scaled
ContestId,,,,,,,,,,,,,,,,,,,,,
24567084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.58,0.18,0.5,0.16,0.44,0.44,0.44,0.44,0.42,0.36


In [4]:
numFail = len(contests.loc[contests['Success']==0])
percent = numFail/len(contests)
1-percent

0.9467517361705039

In [5]:
CLF_contests = contests.loc[:].dropna()

In [6]:
CLF_contests['USE_FOR_BALANCE'] = np.random.uniform(0, 1, len(CLF_contests)) <= percent
CLF_contests = CLF_contests.loc[(CLF_contests["USE_FOR_BALANCE"] == True) | (CLF_contests["Success"] == 0)]
CLF_Features = CLF_contests.columns

In [7]:
CLF_Features= CLF_contests.columns.drop('Success')
CLF_Features = CLF_Features.drop('USE_FOR_BALANCE')

In [8]:
CLF_contests['is_train'] = np.random.uniform(0, 1, len(CLF_contests)) <= .75
train, test = CLF_contests[CLF_contests['is_train']==True], CLF_contests[CLF_contests['is_train']==False]
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 4581
Number of observations in the test data: 1514


In [9]:
actual = test['Success']
y = train['Success']

In [10]:
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(train[CLF_Features], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [11]:
# Create actual english names for the plants for each predicted plant class
preds = clf.predict(test[CLF_Features])
trueIds = test[['Success']]


# View the PREDICTED species for the first five observations
print('Predicted price of first five observations', preds[0:5])
# View the ACTUAL species for the first five observations
print('Actual Price of first five observations', test['Success'].head())

Predicted price of first five observations [0. 1. 0. 0. 1.]
Actual Price of first five observations ContestId
24567084    0.0
24878538    1.0
24878532    0.0
24878890    0.0
24912812    0.0
Name: Success, dtype: float64


In [12]:
accuracy_score(actual, preds)

0.8731836195508587

In [13]:
confuse = pd.DataFrame({'Predict':preds, 'Real':actual})

In [14]:
successes =  len(test.loc[test['Success']==True])
fails = len(test.loc[test['Success']==False])

In [15]:
print("fails:", fails, "---", "successes:", successes)

fails: 791 --- successes: 723


In [16]:
print(confusion_matrix(actual, preds))

[[727  64]
 [128 595]]


In [17]:
# true.columns = ['RealVal']
trueDF = pd.DataFrame(data=actual)
trueDF.columns = ['RealVal']
trueDF = trueDF.assign(PredVal= preds.tolist())


In [18]:
failDF = trueDF.loc[trueDF['RealVal'] != trueDF['PredVal']]
falsePositives = failDF.loc[failDF['PredVal']==1][[]]
falseNegatives = failDF.loc[failDF['PredVal']==0][[]]
falsePositivesDF = pd.merge(falsePositives, rawContests, on='ContestId', how='left')
falseNegativesDF = pd.merge(falseNegatives, rawContests, on='ContestId', how='left')

In [19]:
# falsePositivesDF.to_csv("data/ForJon/PCAFalsePositives.csv")
# falseNegativesDF.to_csv("data/ForJon/PCAFalseNegatives.csv")